In [ ]:
key = "OPEN_AI_KEY"

In [ ]:
!pip install chromadb
!pip install -U langchain-openai  ##Integrates OpenAI's models (like GPT-3.5, GPT-4, and embeddings) into LangChain. Gives access to:
#                                                                                 ChatOpenAI: Chat-based language models (e.g., GPT).
#                                                                               OpenAIEmbeddings: Text-to-vector embeddings using OpenAI models.
!pip install langchain_community
!pip install --upgrade --quiet langchain faiss-cpu openai youtube-transcript-api


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 12.5 MB/s eta 0

In [ ]:
import os
from getpass import getpass
from langchain_community.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [ ]:

os.environ["OPENAI_API_KEY"] = key
embeddings = OpenAIEmbeddings()

def vector_db_youtube(video_url: str):
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)
    db = Chroma.from_documents(docs, embedding=embeddings)
    return db

def get_response_from_query(db, query, k=4):
    docs = db.similarity_search(query, k=k)
    docs_page = " ".join([doc.page_content for doc in docs])

    chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        Use the transcript below to answer the question:
        Transcript: {docs}
        Question: {question}
        If unsure, say "I don't know."
        """
    )
    chain = LLMChain(llm=chat, prompt=prompt)
    return chain.run(question=query, docs=docs_page).strip()



In [ ]:

# 🚀 Main Interaction Loop
print("📺 YouTube Video Q&A")
video_url = input("Paste YouTube video URL: ")

try:
    db = vector_db_youtube(video_url)
    print("✅ Transcript processed!")

    while True:
        query = input("Ask a question (or type 'exit'): ")
        if query.lower() == "exit":
            break
        if not query.strip():
            continue

        answer = get_response_from_query(db, query)
        print("\n💬 Response:\n", answer)
        print("-" * 60)

except Exception as e:
    print("❌ Error:", e)


📺 YouTube Video Q&A
Paste YouTube video URL: https://www.youtube.com/watch?v=FwOTs4UxQS4
✅ Transcript processed!
Ask a question (or type 'exit'): what are AI agents?


<ipython-input-5-1623360509>:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=prompt)
<ipython-input-5-1623360509>:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run(question=query, docs=docs_page).strip()



💬 Response:
 AI agents are applications or systems that have agentic capabilities, such as reasoning and acting autonomously based on predefined paths set by humans. They can perform tasks like identifying objects in video footage, critiquing their own output, and following iterative processes to achieve a desired outcome.
------------------------------------------------------------
Ask a question (or type 'exit'): types of AI agents?

💬 Response:
 I don't know.
------------------------------------------------------------
Ask a question (or type 'exit'): tell me about AI workflows

💬 Response:
 AI workflows are predefined paths set by humans that guide AI systems in completing tasks or processes. These workflows involve a series of steps that the AI system follows to achieve a specific goal or outcome. They can include tasks such as data compilation, summarization, drafting content, and scheduling automated processes. AI workflows are designed to streamline and automate repetitive tas